# Fine-tuning a Language Model for Custom-Style Text Generation

This notebook demonstrates how to fine-tune a language model to generate text in a custom-style voice. We'll use a dataset of paired emails (standard and custom-style) to teach the model how to transform regular text into custom speech.

## Setup and Imports

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Clone the repository
!git clone https://github.com/TheBormann/humanize-LLM.git
!cd humanize-LLM && pip install -r requirements.txt

fatal: destination path 'humanize-LLM' already exists and is not an empty directory.


In [5]:
import os
import logging
import sys
import pandas as pd
from typing import List, Dict

# Add the parent directory to the path
sys.path.append('/content/humanize-LLM')

from src.utils.model_utils import get_local_model
from src.models.base import FinetuningArguments, PEFTArguments

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)
logger = logging.getLogger(__name__)

## Data Loading and Preparation

We'll load our dataset of paired emails from a CSV file. The dataset contains regular emails in the `body_ai` column and their custom-style versions in the `body` column.

In [6]:
def load_emails_from_csv(file_path: str) -> pd.DataFrame:
    """Load emails from a CSV file with semicolon delimiter."""
    df = pd.read_csv(file_path, sep=';')
    logger.info(f"Loaded {len(df)} emails from {file_path}")
    return df

def prepare_training_data(emails_df: pd.DataFrame) -> List[Dict[str, str]]:
    """Prepare training data from emails dataframe.

    Creates direct style transfer pairs where:
    - prompt: [AI-generated email from body_ai]
    - response: [Your styled version from body]
    """
    training_data = []

    for _, row in emails_df.iterrows():
        if pd.isna(row['body']) or pd.isna(row['body_ai']):
            continue

        sample = {
            "prompt": row['body_ai'],  # Direct AI-generated email
            "response": row['body']     # Your styled version
        }

        training_data.append(sample)

    logger.info(f"Created {len(training_data)} direct style transfer pairs")
    return training_data

In [7]:
# Set the path to the CSV file
EMAIL_CSV_PATH = '/content/humanize-LLM/data/manual_emails.csv'

# Load and prepare the dataset
emails_df = load_emails_from_csv(EMAIL_CSV_PATH)
training_data = prepare_training_data(emails_df)

# Display a sample of the training data
if training_data:
    print("Sample training pair:")
    print(f"Prompt (AI-generated): {training_data[0]['prompt'][:150]}...")
    print(f"Response (custom-style): {training_data[0]['response'][:150]}...")
    print(f"Total training pairs: {len(training_data)}")
else:
    print("No training data found or prepared.")

Sample training pair:
Prompt (AI-generated): Hi [Name],\n\nI'm [Your Name], founder of [Startup Name]. We're revolutionizing [industry] through [key innovation]. Would you have time next week to ...
Response (custom-style): Ahoy [Name],\n\nYer lookin' at [Your Name], fearsome captain of [Startup Name]. We be chartin' treacherous waters of [industry] with [key innovation] ...
Total training pairs: 69


## Fine-tuning Function

Now we'll define a function to fine-tune our model using the prepared dataset. We'll use Parameter-Efficient Fine-Tuning (PEFT) with LoRA to efficiently adapt the model to our custom-style text generation task.

In [8]:
def finetune_model(model_name: str, dataset: List[Dict[str, str]],
                  output_dir: str, use_lora: bool = True):
    """Fine-tune model on custom-style email dataset."""
    # Load base model
    model = get_local_model(model_name)

    # Load the model
    model.load_model(model_name)

    # Configure fine-tuning arguments
    ft_args = FinetuningArguments(
        train_data=dataset,
        epochs=3,  # Usually 2-5 epochs works well for style tuning
        batch_size=1,  # Small batch size for memory efficiency
        learning_rate=2e-5  # Lower learning rate to preserve general knowledge
    )

    # Configure LoRA if used
    peft_args = None
    if use_lora:
        peft_args = PEFTArguments(
            method="lora",
            rank=16,  # Higher rank for more expressiveness
            alpha=32,
            dropout=0.05
        )

    # Run fine-tuning
    result = model.finetune(ft_args, output_dir=output_dir, peft_args=peft_args)
    logger.info(f"Fine-tuning result: {result}")

    # Test the model with direct input
    test_prompt = "Dear Team,\n\nI wanted to remind everyone about our quarterly meeting next Tuesday at 2pm. Please bring your project updates and be prepared to discuss next steps.\n\nRegards,\nManager"
    response = model.generate(test_prompt, max_new_tokens=300)

    logger.info(f"Sample output after fine-tuning:\n{response}")

    return model

## Run the Fine-tuning Process

Now let's set up our configuration and run the fine-tuning process. We'll use a small model like Ministral-3b-instruct for faster fine-tuning.

In [ ]:
# Configuration
MODEL_NAME = "ministral/Ministral-3b-instruct"  # For fine-tuning
OUTPUT_DIR = "../output/custom_style_model"

# Create output directories
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Fine-tune the model
# Note: This may take a while depending on your hardware
model = finetune_model(MODEL_NAME, training_data, OUTPUT_DIR, use_lora=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Test the Fine-tuned Model

Let's test our fine-tuned model with some example prompts to see how well it generates custom-style text.

In [ ]:
# Test with different prompts
test_prompts = [
    "Hello, I'm writing to inquire about your services. Could we schedule a call next week?",
    "Dear HR, I'm submitting my application for the software developer position. I have 5 years of experience.",
    "Team, please remember to submit your reports by Friday. The client is expecting our analysis.",
]

for i, prompt in enumerate(test_prompts):
    print(f"\nTest Prompt {i+1}:\n{prompt}")
    response = model.generate(prompt, max_new_tokens=300)
    print(f"\ncustom-Style Response:\n{response}\n")

## Conclusion

In this notebook, we've demonstrated how to fine-tune a language model to generate text in a specific style - in this case, custom-speak. The same approach can be used for other stylistic transformations, such as formal to casual, technical to simple, or any other style you have paired examples for.

Key points:

1. We used direct style transfer pairs (standard → custom) for training
2. We applied LoRA for parameter-efficient fine-tuning
3. We kept the learning rate low to preserve the model's general knowledge
4. We tested the model with various prompts to evaluate its style adaptation

This approach can be extended to personalize AI responses to match your unique voice or brand tone.